# PyTorch 构建神经网络

PyTorch 搭建神经网络结构的组件在 torch.nn 中 ，这些神经网络层大多以类出现，例如全连接层：torch.nn.Linear() ，MSE 损失函数类torch.nn.MSELoss()  等

除此之外，torch.nn.functional  下面同样也有神经网络层，激活函数，损失函数等，但均以函数出现，例如全连接层函数：torch.nn.functional.linear() ，MSE 损失函数：torch.nn.functionalmse_loss()  等。

MNIST 中每个样本是$28×28$的矩阵，目标是字符 0-9。

In [2]:
# 从蓝桥云课服务器下载数据集
!wget -nc --restrict-file-names=nocontrol "http://labfile.oss.aliyuncs.com/courses/1081/MNIST.zip"
!unzip -o "MNIST.zip"

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'unzip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [3]:
import torchvision

train = torchvision.datasets.MNIST(root='.',train=True,transform = torchvision.transforms.ToTensor(),
                                   download=True)
test = torchvision.datasets.MNIST(root='.',train = False,transform=torchvision.transforms.ToTensor(),
                                  download=True)



上面的代码中，transform=torchvision.transforms.ToTensor()  是利用了 torchvision 提供的 transforms 直接将原 NumPy 数组转换为 PyTorch 张量。

In [4]:
train.data.shape, train.targets.shape, test.data.shape, test.targets.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

接下来，我们还需要使用 PyTorch 提供的一个组件对数据进行封装。torch.utils.data.DataLoader  是 PyTorch 提供的及其常用的数据加载器，它可以将数据集封装成迭代器以方便我们后续进行小批量加载，数据打乱等操作。数据加载器准备好之后，后续只需要通过 for 循环来使用即可。



In [5]:
import torch
train_loader = torch.utils.data.DataLoader(dataset = train,batch_size=64,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test,batch_size=64,shuffle=False)

train_loader,test_loader

(<torch.utils.data.dataloader.DataLoader at 0x2bdd8dabe80>,
 <torch.utils.data.dataloader.DataLoader at 0x2bdd8dabf10>)

首先，torch.nn 中的一个基础类 torch.nn.Module 。该类是 PyTorch 中所有神经网络的基类，它既可以表示神经网络中的某层，也可以表示若干层的神经网络。torch.nn 中的各个类实际上就是由 torch.nn.Modules 继承而拓展。所以，在实际使用中，我们可以继承nn.Module，撰写自定义网络层。

所以，当我们搭建神经网络时，也需要继承 torch.nn.Module。我们准备搭建一个包含两个隐含层的全连接网络。

In [6]:
# 输入（784） → 全连接层 1 （784, 512）→ 全连接层 2 （512, 128）→ 输出（10）
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(784,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128, 10)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

我们定义了新的神经网络结构类 Net()，并使用 nn.Linear 组合了 3 个线性层（全连接层）。前向传播过程中，代码使用了 PyTorch 中常用的函数模块 torch.nn.functional 提供的 RELU 激活函数，实际上你也可以通过实例化 nn.Relu 来达到同样的效

In [8]:
model = Net()
model

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [9]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(),lr=0.002)
# 值得注意的是，PyTorch 中优化器需传入模型的参数 model.parameters()，
# 这是 PyTorch 的一个使用特性。

In [10]:
# 训练
def fit(epochs,model,opt):
    print("Start training, please be patient.")
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(train_loader):
            images = images.reshape(-1,28*28)
            labels = labels
            output = model(images)
            loss = loss_fn(output,labels)
            opt.zero_grad()
            loss.backward()
            opt.step
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Batch [{}/{}], Train loss: {:.3f}'
                      .format(epoch+1, epochs, i+1, len(train_loader), 
loss.item()))
                # 每个 Epoch 执行一次测试
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28)
            labels = labels
            outputs = model(images)
            # 得到输出最大值 _ 及其索引 predicted
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()  # 如果预测结果和真实值相等则计数 +1
            total += labels.size(0)  # 总测试样本数据计数
        print('============ Test accuracy: {:.3f} ============='.format(
            correct / total))
                

In [11]:
fit(epochs=1, model=model, opt=opt)  # 训练 1 个 Epoch，预计持续 10 分钟

Start training, please be patient.
Epoch [1/1], Batch [100/938], Train loss: 2.294
Epoch [1/1], Batch [200/938], Train loss: 2.304
Epoch [1/1], Batch [300/938], Train loss: 2.294
Epoch [1/1], Batch [400/938], Train loss: 2.320
Epoch [1/1], Batch [500/938], Train loss: 2.304
Epoch [1/1], Batch [600/938], Train loss: 2.311
Epoch [1/1], Batch [700/938], Train loss: 2.300
Epoch [1/1], Batch [800/938], Train loss: 2.306
Epoch [1/1], Batch [900/938], Train loss: 2.290
============ Test accuracy: 0.097 =============


首先，由于 PyTorch 没有提供类似于 Flatten 这样的展平类，所以我们通过 reshape 操作将输入 
$28×28$ 展平为 784，使其和网络结构参数符合。你也可以使用 view，但官方更推荐使用 reshape 。

其次，opt.zero_grad() 这步非常关键。由于 PyTorch 设计时梯度会累计，所以我们需要手动清零以实现传入一个 Batch，计算梯度，然后更新参数，从而不会因为前面的梯度累计影响后面的参数更新。但 PyTorch 这样设计也是有原因的，比如当我们想提升 Batch 的大小而硬件又无法处理较多数据时，就可以通过梯度累积机制，等待传入多个 Batch 后再更新参数并执行清零，这就给了开发更多的灵活性。同时，后续循环神经网络中也可能利用到这个特性。

# Sequential 容器结构


上面，我们学习了使用 PyTorch 构建神经网络模型的经典方法步骤。你会发现 PyTorch 使用起来比 TensorFlow 要简单一些，主要体现在 DataLoader 数据加载器和前向传播过程调试较为方便，以及无需管理会话等。但是，PyTorch 又似乎比 Keras 要复杂一些，尤其是需要手动构建训练过程，还需要注意执行 opt.zero_grad() 等额外步骤。

实际上，由于 PyTorch 未提供像 tf.keras 这种更高阶的 API，所以无法达到与 Keras 相似的便捷程度。不过，我们可以使用 PyTorch 提供的 Sequential 网络结构来优化上面的经典过程，使得神经网络结构定义的部分更精简一些。

上面，我们通过继承 nn.Module 来定义了网络结构 Net() 类。实际上，利用 nn.Sequential  可以让这个过程更加直观简便。你可以直接按顺序将网络需要的组件类添加到 Sequential 容器结构中。

In [17]:
model_s = nn.Sequential(
    nn.Linear(784,512),
    nn.ReLU(),
    nn.Linear(512,128),
    nn.ReLU(),
    nn.Linear(128,10),)
model_s

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
)

接下来，我们直接利用上面定义好的损失函数和训练函数完成模型优化迭代过程。由于优化器中需要传入模型的参数，所以这里需要修改为后续定义的 Sequential 模型。



In [18]:
opt_s = torch.optim.Adam(model_s.parameters(), lr=0.002)  # Adam 优化器
fit(epochs=1, model=model_s, opt=opt_s)  # 训练 1 个 Epoch

Start training, please be patient.
Epoch [1/1], Batch [100/938], Train loss: 2.316
Epoch [1/1], Batch [200/938], Train loss: 2.302
Epoch [1/1], Batch [300/938], Train loss: 2.311
Epoch [1/1], Batch [400/938], Train loss: 2.303
Epoch [1/1], Batch [500/938], Train loss: 2.295
Epoch [1/1], Batch [600/938], Train loss: 2.298
Epoch [1/1], Batch [700/938], Train loss: 2.316
Epoch [1/1], Batch [800/938], Train loss: 2.305
Epoch [1/1], Batch [900/938], Train loss: 2.309
============ Test accuracy: 0.070 =============


# 使用 GPU 加速训练


In [19]:
torch.cuda.is_available()

True

In [20]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda')

然后修改代码，数据和模型后面添加 .to(dev)，这样 PyTorch 就可以自动判断是否使用 GPU 加速了。首先是对从 DataLoader 中加载出来的每一批次数据后添加 .to(dev)。我们沿用 fit(epochs, model, opt) 中的代码。

In [23]:
def fit(epochs,model,opt):
    print("Start training, please be patient.")
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(train_loader):
            images = images.reshape(-1,28*28).to(dev)
            labels = labels.to(dev)
            outputs = model(images)
            loss = loss_fn(outputs,labels)
            opt.zero_grad()
            loss.backward()
            opt.step()
            if (i+1)%100==0:
                print('Epoch[{}/{}],Batch[{}/{}],Train loss:{:.3f}'.format(epoch+1,epochs,i+1,len(train_loader),loss.item()))
        correct = 0
        total = 0
        for images,labels in test_loader:
            images = images.reshape(-1,28*28).to(dev)
            labels = labels.to(dev)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,1)
            correct +=(predicted==labels).sum().item()
            total += labels.size(0)
        print('============ Test accuracy: {:.3f} ============='.format(
            correct / total))

In [24]:
model_s.to(dev)
opt_s = torch.optim.Adam(model_s.parameters(), lr=0.002)
fit(epochs=1, model=model_s, opt=opt_s)  # 训练 1 个 Epoch

Start training, please be patient.
Epoch[1/1],Batch[100/938],Train loss:0.055
Epoch[1/1],Batch[200/938],Train loss:0.094
Epoch[1/1],Batch[300/938],Train loss:0.206
Epoch[1/1],Batch[400/938],Train loss:0.040
Epoch[1/1],Batch[500/938],Train loss:0.049
Epoch[1/1],Batch[600/938],Train loss:0.106
Epoch[1/1],Batch[700/938],Train loss:0.056
Epoch[1/1],Batch[800/938],Train loss:0.056
Epoch[1/1],Batch[900/938],Train loss:0.133
============ Test accuracy: 0.968 =============


# 模型保存与推理

In [25]:
torch.save(model_s, './model_s.pt') # .pt文件

In [26]:
model_s = torch.load('./model_s.pt')
model_s

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
)

In [27]:
model_s = torch.load('./model_s.pt')
model_s

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
)

In [28]:
# 对测试数据第一个样本进行推理，注意将张量类型转换为 FloatTensor
result = model_s(test.data[0].reshape(-1, 28*28).type(torch.FloatTensor).to(dev))
torch.argmax(result)  # 找到输出最大值索引即为预测标签

tensor(7, device='cuda:0')

In [29]:
test.targets[0]  # 第一个测试数据真实标签

tensor(7)